# Hartree-Fock VQE Demo

In [1]:
import tqdm

import numpy as np
import scipy as sp
import torch

from QuICT.algorithm.quantum_machine_learning.model.chemistry.utils.hamiltonian.moleculardata import MolecularData
from QuICT.algorithm.quantum_machine_learning.model.chemistry.utils.hamiltonian import obi_basis_rotation, tbi_basis_rotation, generate_hamiltonian
from QuICT.algorithm.quantum_machine_learning.model.chemistry.utils.operators.encoder import JordanWigner
from QuICT.algorithm.quantum_machine_learning.model.chemistry import HartreeFockVQE, HartreeFockVQENet
from QuICT.algorithm.quantum_machine_learning.utils import Hamiltonian

/home/chenziheng/base/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load the data

In [2]:
moldir = "./utils/hamiltonian/molecular_data/hydrogen_chains/h_6_sto-3g/bond_distance_1.3"
molfile = moldir + "/H6_sto-3g_singlet_linear_r-1.3.hdf5"
moldata = MolecularData(molfile)

overlap = np.load(moldir + "/overlap.npy")
Hcore = np.load(moldir + "/h_core.npy")
two_electron_integral = np.einsum("psqr", np.load(moldir + "/tei.npy"))  # (1, 1, 0, 0)

_, X = sp.linalg.eigh(Hcore, overlap)
obi = obi_basis_rotation(Hcore, X)
tbi = tbi_basis_rotation(two_electron_integral, X)
molecular_hamiltonian = generate_hamiltonian(moldata.nuclear_repulsion, obi, tbi)

## Convert the Hamiltonian

In [3]:
fermi_op = molecular_hamiltonian.get_fermion_operator()
orbitals = 2 * moldata.n_orbitals
electrons = moldata.n_electrons
qubit_op = JordanWigner(orbitals).encode(fermi_op)
hamiltonian = Hamiltonian(qubit_op.to_hamiltonian())

## Calculate the ground energy

In [4]:
MAX_ITERS = 1000
LR = 0.1

hfvqe_net = HartreeFockVQENet(orbitals, electrons, hamiltonian)
optim = torch.optim.Adam([dict(params=hfvqe_net.parameters(), lr=LR)])
scheduler = torch.optim.lr_scheduler.StepLR(optim, step_size=100, gamma=0.1)
energy = []

hfvqe_net.train()
loader = tqdm.trange(MAX_ITERS, desc="Training", leave=True)
for it in loader:
    optim.zero_grad()
    state = hfvqe_net()
    loss = hfvqe_net.loss_func(state)
    energy.append(float(loss))
    loss.backward()
    optim.step()
    scheduler.step()
    loader.set_postfix(loss=loss.item())

print(energy)
print(hfvqe_net.params)
print(float(loss))

Training: 100%|██████████| 1000/1000 [52:49<00:00,  3.17s/it, loss=-2.96]

[-2.3692975365068394, -2.3475760648015154, -2.3692954648376094, -2.356178198977554, -2.356220023516117, -2.3676932145763674, -2.3690334184473016, -2.3742920342811766, -2.4157439480266563, -2.4840887040872004, -2.5331097876311697, -2.5679207277977634, -2.580035373906412, -2.5878767885861818, -2.5977833848689973, -2.613802968174456, -2.6274047940190406, -2.6248650771643716, -2.626617888072876, -2.6509343281825393, -2.6792258575108434, -2.6908132289790574, -2.686883820802344, -2.6806725453823397, -2.6850396077570915, -2.6972917609843483, -2.7085418328666258, -2.7166358190134057, -2.725381607282556, -2.7328277816356312, -2.734316296425526, -2.738077638241547, -2.750111605832208, -2.763694327527646, -2.7743539160226582, -2.7836781062917892, -2.791832591586589, -2.7967753786437197, -2.7986913014302504, -2.80030199755293, -2.8034246309812336, -2.8097861045314256, -2.8201649609702595, -2.835475422573836, -2.853547590233567, -2.869440961789394, -2.881368018839158, -2.887902826066643, -2.8926346